# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
df = pd.read_csv("Hasil_Cluster.csv")
df.head()

,BHK,Rent,Size,Bathroom,Cluster,City,Area Type
0,2.0,10000.0,1100.0,2.0,1,Kolkata,Super Area
1,2.0,20000.0,800.0,1.0,1,Kolkata,Super Area
2,2.0,17000.0,1000.0,1.0,1,Kolkata,Super Area
3,2.0,10000.0,800.0,1.0,1,Kolkata,Super Area
4,2.0,7500.0,850.0,1.0,2,Kolkata,Carpet Area


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['City', 'Area Type']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

categorical_encoded = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(categorical_cols))

df = df.drop(columns=categorical_cols).reset_index(drop=True)
df = pd.concat([df, encoded_df], axis=1)

In [4]:
y = df['Cluster']
X = df.drop('Cluster', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")

X_train: (3796, 13)
X_test: (950, 13)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [5]:
rf_clf = RandomForestClassifier(random_state=42)
dt_clf = DecisionTreeClassifier(random_state=42)

rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)
y_pred_dt = dt_clf.predict(X_test)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [6]:
def evaluate_model(y_test, y_pred, model_name):
    print(f"\nEvaluasi {model_name}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
    print(classification_report(y_test, y_pred))

evaluate_model(y_test, y_pred_rf, "Random Forest")
evaluate_model(y_test, y_pred_dt, "Decision Tree")


Evaluasi Random Forest:
Accuracy: 1.0000
F1-Score: 1.0000
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       356
           2       1.00      1.00      1.00       242
           3       1.00      1.00      1.00       167
           4       1.00      1.00      1.00       132
           5       1.00      1.00      1.00        53

    accuracy                           1.00       950
   macro avg       1.00      1.00      1.00       950
weighted avg       1.00      1.00      1.00       950


Evaluasi Decision Tree:
Accuracy: 1.0000
F1-Score: 1.0000
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       356
           2       1.00      1.00      1.00       242
           3       1.00      1.00      1.00       167
           4       1.00      1.00      1.00       132
           5       1.00      1.00      1.00        53

    accuracy                           1.00       950
   macro avg 

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [7]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [8]:
def tune_model(model, param_grid, X_train, y_train, skf):
    grid_search = GridSearchCV(model, param_grid, cv=skf, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
best_rf_clf = tune_model(RandomForestClassifier(random_state=42), param_grid_rf, X_train, y_train, skf)

param_grid_dt = {'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
best_dt_clf = tune_model(DecisionTreeClassifier(random_state=42), param_grid_dt, X_train, y_train, skf)

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [9]:
def evaluate_tuned_model(y_test, best_model, model_name):
    y_pred_best = best_model.predict(X_test)
    print(f"\nHasil Evaluasi {model_name} :")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_best):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred_best, average='weighted'):.4f}")
    print(classification_report(y_test, y_pred_best))
evaluate_tuned_model(y_test, best_rf_clf, "Random Forest")
evaluate_tuned_model(y_test, best_dt_clf, "Decision Tree")


Hasil Evaluasi Random Forest :
Accuracy: 1.0000
F1-Score: 1.0000
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       356
           2       1.00      1.00      1.00       242
           3       1.00      1.00      1.00       167
           4       1.00      1.00      1.00       132
           5       1.00      1.00      1.00        53

    accuracy                           1.00       950
   macro avg       1.00      1.00      1.00       950
weighted avg       1.00      1.00      1.00       950


Hasil Evaluasi Decision Tree :
Accuracy: 1.0000
F1-Score: 1.0000
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       356
           2       1.00      1.00      1.00       242
           3       1.00      1.00      1.00       167
           4       1.00      1.00      1.00       132
           5       1.00      1.00      1.00        53

    accuracy                           1.00       950

In [10]:
# Cross-validation
def cross_val_performance(model, X, y, skf):
    cross_val_accuracy = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
    cross_val_f1 = cross_val_score(model, X, y, cv=skf, scoring='f1_weighted')
    return cross_val_accuracy, cross_val_f1

# Cross-validation untuk masing-masing model
models = [best_rf_clf, best_dt_clf]
model_names = ["Random Forest", "Decision Tree"]

for model, name in zip(models, model_names):
    cross_val_accuracy, cross_val_f1 = cross_val_performance(model, X, y, skf)
    print(f"\nCross-validation untuk {name}:")
    print(f"Accuracy: {cross_val_accuracy.mean():.4f} ± {cross_val_accuracy.std():.4f}")
    print(f"F1-Score: {cross_val_f1.mean():.4f} ± {cross_val_f1.std():.4f}")


Cross-validation untuk Random Forest:
Accuracy: 1.0000 ± 0.0000
F1-Score: 1.0000 ± 0.0000

Cross-validation untuk Decision Tree:
Accuracy: 1.0000 ± 0.0000
F1-Score: 1.0000 ± 0.0000


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

### 1. Perbandingan Hasil Evaluasi Sebelum dan Setelah Tuning

Berdasarkan hasil evaluasi yang diberikan, baik model **Random Forest** maupun **Decision Tree** menunjukkan akurasi sempurna dengan nilai **Accuracy = 1.0000** dan **F1-Score = 1.0000** untuk semua kelas. Jika tuning telah dilakukan, tampaknya tidak ada perubahan signifikan dalam hasil akhir, yang mengindikasikan bahwa model telah mencapai performa maksimal dalam dataset ini.

Selain itu, hasil **cross-validation** juga menunjukkan **Accuracy = 1.0000 ± 0.0000** dan **F1-Score = 1.0000 ± 0.0000** untuk kedua model. Ini menandakan bahwa model tetap memberikan hasil yang sempurna di berbagai subset data.

### 2. Identifikasi Kelemahan Model

#### Precision dan Recall Tinggi untuk Semua Kelas
Setiap kelas memiliki **precision, recall, dan F1-score yang sempurna (1.00)**, yang menandakan tidak ada kesalahan klasifikasi dalam data uji.

#### Potensi Overfitting
Hasil yang terlalu sempurna sering kali menjadi tanda bahwa model mengalami **overfitting**.

#### Dataset Kemungkinan Terlalu Mudah atau Tidak Beragam
Jika dataset tidak cukup kompleks atau memiliki pola yang sangat jelas, model dapat dengan mudah mencapai akurasi sempurna.

### 3. Rekomendasi Tindakan Lanjutan

Untuk memastikan bahwa model ini benar-benar dapat digunakan dalam dunia nyata tanpa risiko overfitting, beberapa langkah dapat dilakukan:

#### Periksa Kualitas Dataset
- Apakah ada data yang terlalu bersih atau berulang?

#### Lakukan Validasi Tambahan
- Coba evaluasi model dengan dataset baru atau yang belum pernah digunakan dalam pelatihan.

#### Eksperimen dengan Model Lain
- Jika ingin memastikan hasil lebih **robust**, bisa mencoba model lain seperti **Logistic Regression, SVM, atau Neural Networks**.

#### Tambahkan Data Baru Jika Diperlukan
- Jika data yang digunakan tidak cukup kompleks, pertimbangkan untuk mengumpulkan atau menambahkan data baru yang lebih beragam agar model bisa diuji dengan skenario yang lebih realistis.
